In [56]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import torch
import os

In [57]:
#mnist数据集是灰度图，分别是均值和标准差，操作后具有均值为0，标准差为1的特性
Fashion_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.286,), (0.353,))])
transform_to_tensor = transforms.ToTensor()
#加载数据集
data_train = datasets.FashionMNIST(root='./data', train=True, download=True, transform=Fashion_transforms)
data_test = datasets.FashionMNIST(root='./data', train=False, download=True, transform=Fashion_transforms)
#加载dataloader
dataloader_train = DataLoader(dataset=data_train, batch_size=64, shuffle=True, drop_last=True)  #28x28
dataloader_test = DataLoader(dataset=data_test, batch_size=64, shuffle=True, drop_last=True)
#FashionMnist由图片和类别索引构成

In [58]:
#用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda is available?", torch.cuda.is_available())

cuda is available? True


In [59]:
#网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1)  #灰度图像，通道数为1，有32个输出通道，也就是有32个卷积核产生图像，输出26x26
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  #第二层卷积，通道数为32，输出64，输出完变成
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(13 * 13 * 64, 10)  #返回十个数

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = x.view(-1, 13 * 13 * 64)
        #将池化层的输出x转换成一个一维数组，然后将其重新排列（reshape）成一个二维数组，其中第二维的大小是13*13*64，这里的13*13是池化后的特征图尺寸，64是通道数，-1表示第一维的大小由第二维的大小推导得出
        x = self.fc1(x)
        #x = F.softmax(x, dim=1)
        return x

In [60]:
#模型
model = Net().to(device)
#优化器
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  #weight_decay施加L2正则化
#加载
if os.path.exists("./FashionModel/model.pth"):
    model.load_state_dict(torch.load('./FashionModel/model.pth'))  # 加载保存的模型
    optimizer.load_state_dict(torch.load('./FashionModel/optimizer.pth'))  # 加载保存的优化器，主要是学习率
    print("load model successfully")

load model successfully


In [61]:
#F1分数
def f1_score(pred, targets):
    beta = 1  # 通常使用F1分数时，beta为1
    pred = pred.cpu().numpy()  # 确保预测结果在CPU上
    targets = targets.cpu().numpy()
    tp = np.sum((pred == 1) & (targets == 1))
    fp = np.sum((pred == 1) & (targets == 0))
    fn = np.sum((pred == 0) & (targets == 1))

    if (tp + fp) == 0 or (tp + fn) == 0:
        return 0.0#如果分母为0，返回0

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (1 + beta ** 2) * precision * recall / (beta ** 2 * precision + recall)
    return f1,precision,recall

In [62]:
#训练
def train(train_best_loss):
    model.train()#训练模式
    dataloader = dataloader_train
    for idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # 移动数据到 GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, target=labels)
        loss.backward()
        optimizer.step()
        if loss < train_best_loss:
            train_best_loss = loss
            print("idx:", idx, "current best loss:", train_best_loss.item())
            torch.save(model.state_dict(), "./FashionModel/model.pth")
            torch.save(optimizer.state_dict(), "./FashionModel/optimizer.pth")
    return train_best_loss

In [63]:
def test():
    model.eval()  #评估模式
    data_loader = dataloader_test
    acc_list = []
    f1_list=[]#f1分数
    precision_list=[]#精准度:指模型认为的正类别中，有多少个是真正的正类别
    recall_list=[]#召回率:在全部的正类别中，找到了多少个正类别
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # 确保数据在 GPU
            outputs = model(inputs)
            # 计算准确率
            # output[batch_size,10] target[batch_size]
            # 输出每一行都有十个数，代表对应0-9的概率，找出最大的概率代表着模型识别出来的值
            pred = torch.max(outputs, dim=1)[1]#模型预测值
            pred = pred.cpu()
            cur_acc = pred.eq(labels.cpu()).float().mean().item()
            acc_list.append(cur_acc)
            
            #计算f1
            cur_f1,cur_precision,cur_recall=f1_score(pred.cpu(),labels.cpu())
            f1_list.append(cur_f1)
            precision_list.append(cur_precision)
            recall_list.append(cur_recall)
    print("平均准确率:", np.mean(acc_list))
    print("平均F1分数:", np.mean(f1_list))
    print("平均精准度:",np.mean(precision_list))
    print("平均召回率:",np.mean(recall_list))

In [64]:
best_loss = float('inf')
for i in range(1):
    best_loss = train(train_best_loss=best_loss)
test()

idx: 0 current best loss: 0.12893064320087433
idx: 1 current best loss: 0.05359765887260437
idx: 46 current best loss: 0.05209989845752716
idx: 50 current best loss: 0.03893166780471802
idx: 72 current best loss: 0.02147693745791912
idx: 75 current best loss: 0.020870789885520935
平均准确率: 0.9026442307692307
平均F1分数: 0.9969567469567471
平均精准度: 0.9987179487179488
平均召回率: 0.9957264957264957
